# Задание по многопоточности:

Вам необходимо проанализировать википедию на предмет того, какие слова в каждой из частей речи встречаются чаще. Вы хотите реализовать это в несколько потоков.

Запросы к википедии можно осуществлять с помощью библиотеки wikipedia. Для морфологического анализа используйте библиотеку pymorphy2. Чтобы разбить текст на слова можете воспользоваться функцией word_tokenize из библиотеки nltk.

Класс должен иметь функции, приведенные ниже (но может иметь и другие на ваше усмотрение).

In [1]:
def bool_check(b):
    b[0] = True

In [2]:
b = [False]
bool_check(b)
print(b)

[True]


### Часть 1

<b>Многопоточной реализация</b>

Задачи делятся на три типа:
<ul>
<li><i>Получение данных</i>:
<ol>
<li>Получение заголовков для страниц википедии - запускает по max_threads функций, которые асинхронно получают заголовки страниц.</li>
<li>Получение конкретных страниц - ждем, пока не появятся новые заголовки, которые не обработаны.
Когда появились - начинаем запрашивать в max_threads функциях конкретные страницы по заголовкам.</li>
</ol>
</li>
<li><i>Обработка данных</i>:
<ol>
<li>Ждем, пока не появятся новые необработанные страницы. Когда появляются, запускаем по max_threads функций для морфологического анализа слов.</li>
<li>Ждем пока не появились обработанные слова. Как только появляется новое слово, сразу же обновляем _stats.</li>
</ol>
</li>
<li><i>Сохранение данных</i>:
<ol>
        Раз в store_every обработанных слов вызывается асинхронно функция dump, которая сохраняет _stats.
</ol>
</li>
</ul>
<b>P. S.</b>

Комментарии специально запутанные, чтобы вы сами придумали архитектуру вызова потоков. Не бойтесь использовать Queue и daemon=True. Запрещается использовать threading.Lock / threading.RLock или другие блокировки.

In [3]:
from collections import defaultdict, Counter
from queue import Queue

import numpy as np
import threading
import json
import wikipedia
import nltk
import pymorphy2

wikipedia.languages()

wikipedia.set_lang('ru')

title = wikipedia.random(pages=1)

print(title)

page = wikipedia.page(title)

page.content

text = page.content
words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

words[:8]

morph = pymorphy2.MorphAnalyzer()

p = morph.parse(words[0])[0]
p.tag.POS

type(morph.parse(words[6])[0].tag.POS)

np.array(wikipedia.random(pages=1)).reshape((-1))

for title in np.array(wikipedia.random(pages=2)).reshape((-1)):
    print(title)

In [12]:
class WikiReader(object):
    """
    Класс для работы с википедией.
    Собирает статисткику по словам каждой части речи в статьях википедии.

    Parameters
    ----------
    morphs: list
        Части речи, которые хотим исследовать. Слова другой части речи не включаются в статистику.
    
    page_per_req: int
        Количество случайных названий страниц, запрашиваемых за один раз у википедии.
    
    max_threads: int
        Количество потоков, запускаемых другим потоком демоном (можно не использовать, если получится).
    
    max_words: int
        Количество слов для обработки.
    
    store_every: int
        Как часто сохранять данные на диск. Каждые store_every слов вызывается функция dump.
    
    store_path: str
        Куда сохранять данные.
    
    Attributes
    ----------
    _stats: <your code here>
        Структура данных (возможно встроенная), позволяющая хранить для каждой части речи список слов с их количеством.
        Необходимо, чтобы получение (изменение) статистики (текущего количества) для каждой пары
        <часть речи, слово> происходило за O(1).
            
    """
    def __init__(self,
                 morphs=[],
                 page_per_req=4,
                 max_threads_per_daemon=8,
                 max_words=10000,
                 store_every=1000,
                 store_path=""):
        self._stats = defaultdict(Counter)
        self.morphs = morphs
        self._stats.update({m: Counter() for m in morphs})
        self.processed_words = 0
        self.dump_count = 0
        self.max_words = max_words
        self.store_every = store_every
        self.store_path = store_path
        self.page_per_req = page_per_req
        self.max_threads_per_daemon = max_threads_per_daemon
        self.run_flags = {}
    
    def run(self):
        wikipedia.set_lang('ru')
        
        titles_q = Queue()
        pages_q = Queue()
        self.run_flags = {
            'title_daemon': True,
            'page_daemon': True,
            'processor_daemon': True,
        }
        
        titles_adder_daemon = threading.Thread(
            target=self.titles_adder_daemon, args=(titles_q,), daemon=True
        )
        titles_adder_daemon.start()

        pages_adder_daemon = threading.Thread(
            target=self.pages_adder_daemon, args=(titles_q, pages_q), daemon=True
        )
        pages_adder_daemon.start()

        processors_adder_daemon = threading.Thread(
            target=self.processors_adder_daemon, args=(pages_q,), daemon=True
        )
        processors_adder_daemon.start()
        
        while self.processed_words < self.max_words:
            pass
        
        self.run_flags = {
            'title_daemon': False,
            'page_daemon': False,
            'processor_daemon': False,
        }
            
        
    def dump(self, path=None):
        self.dump_count += 1
        cur_stats = self._stats.copy()
        with open(path, 'w', encoding="utf-8") as file:
            # writing
            json.dump(cur_stats, file, indent=4, ensure_ascii=False)
    
    def load(self, path=None):
        with open(path, 'r') as file:
            self._stats = json.load(file)
            
    def titles_adder_daemon(self, titles_q):
        while True:
            if self.run_flags.get('title_daemon', False):
                threads = [threading.Thread(target=self.thread_titles_adder, args=(titles_q,))
                           for _ in range(self.max_threads_per_daemon)]
                WikiReader.do_work(threads)
                
    def thread_titles_adder(self, titles_q):
#         while True:
        for title in np.array(wikipedia.random(pages=self.page_per_req)).reshape((-1)):
            titles_q.put(title)
                
            
    def pages_adder_daemon(self, titles_q, pages_q):
        while True:
            if self.run_flags.get('page_daemon', False):
                threads = [threading.Thread(target=self.thread_pages_adder, args=(titles_q, pages_q))
                           for _ in range(self.max_threads_per_daemon)]
                WikiReader.do_work(threads)
        
    def thread_pages_adder(self, titles_q, pages_q):
#         while True:
        title = titles_q.get()
        try:
            pages_q.put(wikipedia.page(title).content)
        except wikipedia.exceptions.DisambiguationError:
            pass
        titles_q.task_done()
                
            
    def processors_adder_daemon(self, pages_q):
        while True:
            if self.run_flags.get('processor_daemon', False):
                threads = [threading.Thread(target=self.thread_page_processing, args=(pages_q,))
                           for _ in range(self.max_threads_per_daemon)]
                WikiReader.do_work(threads)
                
    def thread_page_processing(self, pages_q):
        morph = pymorphy2.MorphAnalyzer()
#         while True:
        page = pages_q.get()
        words = [word for sent in nltk.sent_tokenize(page) for word in nltk.word_tokenize(sent)]
        for word in words:
            self.word_processing(word, morph)
#         list(map(lambda word: self.word_processing(word, morph), words))
        pages_q.task_done()

    def word_processing(self, word, morph):
        p = morph.parse(word)[0].tag.POS
        if p in self.morphs:
            self._stats[p][word] += 1
            self.processed_words += 1
            if self.processed_words >= self.store_every * (self.dump_count + 1):
                self.dump(self.store_path)
                    
    @staticmethod
    def do_work(threads):
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

In [13]:
store_path = "./stat.json"
morphs=['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 
        'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ']
reader = WikiReader(
morphs=morphs, page_per_req=4, max_threads_per_daemon=8,
max_words=10000, store_every=1000, store_path=store_path
)
reader.run()

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
Exception in thread Thread-77:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threadi

In [14]:
a = 1
print(a)

1


In [15]:
reader._stats

defaultdict(collections.Counter,
            {'ADJF': Counter({'Сверхновая': 1,
                      'сверхновых': 4,
                      'которые': 9,
                      'свою': 7,
                      'катаклизмических': 1,
                      'переменных': 1,
                      'белого': 2,
                      'Белый': 1,
                      'которая': 8,
                      'свой': 3,
                      'нормальный': 1,
                      'жизненный': 1,
                      'которой': 8,
                      'термоядерные': 1,
                      'определённых': 1,
                      'дальнейшие': 1,
                      'углеродно-кислородного': 1,
                      'огромное': 1,
                      'своей': 4,
                      'солнечных': 1,
                      'максимальная': 1,
                      'белый': 3,
                      'другие': 4,
                      'спиральная': 1,
                      'всё': 1,
               